***
# `Project:` Prevendo a <font color='blue'>morte</font> ou <font color='blue'>vida</font> de pacientes com hepatite

## `Date:` fevereiro, 2022

## `Data Scientist:` Walter Trevisan
***

<a name='notebook-header'></a>
## `Modelagem Preditiva` (*`Machine Learning`*) - [Decision Tree Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

Neste **notebook** vamos realizar a **modelagem preditiva** treinando e analisando alguns modelos preditivos criados com o algoritmo **Decision Tree Classifier**. Para fazermos o treinamento dos modelos utilizaremos os *data sets* de treino e para avaliarmos a performance dos modelos utilizaremos o *data set* de teste que foram criados e preparados na etapa anterior (`Data Munging`).

### Conteúdo
1. [Setup Inicial](#initial-setup)

2. [Carregar os *dataframes* de treino e de teste](#load-data)

3. [Treinar e Avaliar os modelos preditivos](#modelos-treinar-avaliar)

4. [Trabalhando com **seleção de variáveis** nos melhores modelos preditivos](#modelos-selecao-variaveis)

5. [Concluir e salvar o **melhor modelo preditivo** construído com a **Decision Tree Classifier**](#modelos-conclusao)

___
<a name='initial-setup'></a>
## <font color='blue'>1- Setup Inicial:</font>

Primeiro, vamos carregar os **pacotes e funções** que serão utilizadas neste **notebook**.

In [1]:
# As novas versões do Pandas e Matplotlib trazem diversas mensagens de aviso ao desenvolvedor.
# Então, vamos desativar essas mensagens.
import sys # O pacote "sys" permite manipulações com o sistema operacional:
import os  # Operation System (Packages and Functions)
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Importa função para verificarmos a versão da linguagem python:
from platform import python_version

# Importando os pacote NumPy:
import numpy as np
# Importando os pacote Pandas:
import pandas as pd

# Importando pacotes para visualização de gráficos:
import matplotlib as mpl
import matplotlib.pyplot as plt
# Importa o pacote "seaborn" para criarmos gráficos estatísticos:
import seaborn as sns
%matplotlib inline

# Machine Learning imports
# Importando o pacote do Scikit-Learn:
import sklearn as skl
# Função para padronização de variáveis:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
# Importando algoritmos de classificação:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV

# Definindo uma "semente" para reproduzir os mesmos dados nas tarefas de amostragem,
# balanceamento dos dados e treinamento dos modelos preditivos:
SEED = 42

# Nota: Como vamos equilibrar nossos dados de treinamento, através do "balanceamento" de classes,
# vamos definir nosso limite (threshold) em "0.5" para rotular uma amostra prevista como positiva.
# Portanto, as probabilidades previstas acima deste valor (THRESHOLD) serão rotuladas como positiva
# (target=1), ou seja, significa que os pacientes com hepatite sobreviveram (LIVE).
THRESHOLD = 0.5

# Define valor para "Cross Validation" (Número de 'folds'):
NUM_FOLDS=10 # Número de passadas ("folds")

# Definindo o diretório raiz (Root) onde serão armazenados todas as informações
# (Imagens, Gráficos, Objetos, Dados, Modelos de ML, etc...) do projeto.
# Diretório Raiz (Root) do Projeto:
ROOT_DIR = "."

# Path: onde ficarão armazenados os "Objetos" (Estrututras de Dados) relacionados ao Projeto:
OBJ_PATH = os.path.join(ROOT_DIR, "objects")
# Criando o diretório, se ele não existir:
os.makedirs(OBJ_PATH, exist_ok=True)

# Path: onde ficarão armazenados os "datasets" (arquivos "csv") e os "objetos" (Data Frames) do Projeto:
DATA_PATH = os.path.join(ROOT_DIR, "data")
# Criando o diretório, se ele não existir:
os.makedirs(DATA_PATH, exist_ok=True)

# Path: onde serão armazenadas as "Imagens" (Figuras e Gráficos) do Projeto:
GRAPHICS_PATH = os.path.join(ROOT_DIR, "images", "graphics")
# Criando o diretório, se ele não existir:
os.makedirs(GRAPHICS_PATH, exist_ok=True)

# Path: onde ficarão armazenados os "Modelos Preditivos" (Machine Learning) relacionados ao Projeto:
ML_PATH = os.path.join(ROOT_DIR, "models")
# Criando o diretório, se ele não existir:
os.makedirs(ML_PATH, exist_ok=True)

# Path: onde estão armazenadas as classes e funções que serão utilizadas neste notebook:
LIB_PATH = os.path.join(ROOT_DIR, "library")

# Adicionando o diretório "./library" ao 'path' do Sistema, para podermos importar classes e funções que serão
# utilizadas neste notebook:
sys.path.append(LIB_PATH)

# Importando para este notebook, as classes e funções definidas no módulo "data_science_library.py":
import data_science_library as dslib

# Importando para este notebook, as classes e funções definidas no módulo "plot_library.py":
import plot_library as ptlib

# Importando para este notebook, as classes e funções definidas no módulo "machine_learning_library.py":
import machine_learning_library as mllib

# Criando um objeto para calularmos os tempos gastos de treinamento:
ept = dslib.ElapsedTime(builder_msg=False)

print("Setup Complete!")

Setup Complete!


In [2]:
# Versões dos pacotes usados neste jupyter notebook:
print("Versões dos pacotes usados neste jupyter notebook:")
print("Python      : {}".format(python_version()))
print("Numpy       : {}".format(np.__version__))
print("Pandas      : {}".format(pd.__version__))
print("Matplotlib  : {}".format(mpl.__version__))
print("Seaborn     : {}".format(sns.__version__))
print("Scikit-Learn: {}".format(skl.__version__))

Versões dos pacotes usados neste jupyter notebook:
Python      : 3.8.12
Numpy       : 1.19.5
Pandas      : 1.3.5
Matplotlib  : 3.4.3
Seaborn     : 0.11.2
Scikit-Learn: 1.0.2


___
<a name='load-data'></a>
## <font color='blue'>2- Carregar os *data frames* de `treino` e `teste`</font>

### Dados de Treino:

* `train_set_v1`: nesta versão apenas apliquei **encoding** nas variáveis categóricas;

* `train_set_v2`: nesta versão também apliquei **encoding** nas variáveis categóricas; e tratei e **removi os outliers** existentes nas variáveis numéricas.

In [3]:
# Criando um objeto para armazenar as versões dos dataframes de treino:
train_set = {}

In [4]:
# Carregando o dataframe de treino "train_set_v1":
train_set['v1'] = dslib.pickle_object_load(path=DATA_PATH, file="train_set_v1.pkl")
# Carregando o dataframe de treino "train_set_v2":
train_set['v2'] = dslib.pickle_object_load(path=DATA_PATH, file="train_set_v2.pkl")

### Dados de Teste:

* `test_set`: tratei os valores ausentes nas variáveis categóricas e numéricas; apliquei **encoding** nas variáveis categóricas.

In [5]:
# Carregando o dataframe de teste "test_set":
test_set = dslib.pickle_object_load(path=DATA_PATH, file="test_set.pkl")

Para retornar ao ínicio deste **notebook** clique **[aqui](#notebook-header)**.

___
<a name='modelos-treinar-avaliar'></a>
## <font color='blue'>3- Treinar e Avaliar os modelos preditivos</font>

### Criando uma instância do algoritmo `DecisionTreeClassifier` com os hiperparâmetros padrão:

In [6]:
# Criando uma instância do classificador que será treinado a avaliado em cada versão:
clf = DecisionTreeClassifier(random_state=SEED)
# Label do modelo:
model_label = 'DTC'

### Criando um dicionário para armazenar os `modelos` treinados e avaliados em cada `versão`:

In [7]:
# Criando um dicionário (objeto) para armazenar os "modelos" preditivos treinados e avaliados de cada versão:
models = {}

### Definindo as variáveis preditoras `categóricas`, `numéricas` e a variável `target`:

In [8]:
# Definindo a variável "target" (Class):
target_variable = 'Class'
# Definindo as variáveis categóricas preditoras:
cat_variables = ['Gender', 'Steroid', 'Antivirals', 'Fatigue', 'Malaise', 'Anorexia', 'LiverBig', 'LiverFirm',
                 'SpleenPalpable', 'Spiders', 'Ascites', 'Varices', 'Histology']
# Definindo as variáveis numéricas preditoras:
num_variables = ['Age', 'Bilirubin', 'AlkPhosphate', 'SGOT', 'Albumin', 'Protime']

### Criando um dicionário para armazenar as `features` que serão utilizadas em cada versão: 

In [9]:
# Variáveis preditoras:
features = {}

### Criando um dicionário para armazerarmos o `scaler` aplicado aos dados de `treino`e `teste` em cada versão:

In [10]:
# Scaler:
scaler = {}

### Definindo um *dataframe* para armazenar as métricas de `treino` e `teste`:

In [11]:
# Métricas nos dados de treino:
train_mtcs = {}
# Métricas nos dados de teste:
test_mtcs = {}

In [12]:
# Criando o dataframe para armazenar as métricas de cada versão:
metrics = pd.DataFrame(columns=['Version', 'Model', 'Data set', 'AUC', 'Accuracy', 'Precision', 'Recall', 'F1_score'])
metrics

,Version,Model,Data set,AUC,Accuracy,Precision,Recall,F1_score


In [13]:
# Criando o dataframe para armazenar as métricas da melhor versão de cada algoritmo:
models_best_metrics = dslib.pickle_object_load(path=ML_PATH, file="models_best_metrics.pkl")

### Definindo as `versões` dos modelo preditivos que serão treinados e avaliados:

In [14]:
# Definindo um dicionário dos modelos que serão criados com os parâmetros de cada versão:
models_version = {
    'v01':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':None,'balance':False,'random_state':None},
    'v02':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':StandardScaler(),'balance':False,'random_state':None},
    'v03':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':MinMaxScaler(),'balance':False,'random_state':None},
    'v04':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':RobustScaler(),'balance':False,'random_state':None},
    'v05':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':None,'balance':True,'random_state':SEED},
    'v06':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':StandardScaler(),'balance':True,'random_state':SEED},
    'v07':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':MinMaxScaler(),'balance':True,'random_state':SEED},
    'v08':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':RobustScaler(),'balance':True,'random_state':SEED},
    'v09':{'train_set':'v2','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':None,'balance':False,'random_state':None},
    'v10':{'train_set':'v2','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':StandardScaler(),'balance':False,'random_state':None},
    'v11':{'train_set':'v2','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':MinMaxScaler(),'balance':False,'random_state':None},
    'v12':{'train_set':'v2','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':RobustScaler(),'balance':False,'random_state':None},
    'v13':{'train_set':'v2','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':None,'balance':True,'random_state':SEED},
    'v14':{'train_set':'v2','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':StandardScaler(),'balance':True,'random_state':SEED},
    'v15':{'train_set':'v2','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':MinMaxScaler(),'balance':True,'random_state':SEED},
    'v16':{'train_set':'v2','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':RobustScaler(),'balance':True,'random_state':SEED},
}

### Treinando a avaliando cada `versão` do modelo preditivo:

In [15]:
# Inicia o treinamento e avaliação de cada modelo:
ept.start(msg="Iniciando o treinamento e avaliação de cada versão do modelo...")
# Loop para treinar e avaliar cada versão do modelo:
for v, p in models_version.items():
    # Preparando os dados de "treino" (X_train e y_train) e "teste" (X_test e y_test):
    X_train, y_train, X_test, y_test, scaler[v] = mllib.prepare_train_test_data(
    train_set=train_set[p['train_set']], test_set=test_set, target=target_variable, cat_features=p['cat_variables'],
    num_features=p['num_variables'], scaler=p['scaler'], balance=p['balance'], random_state=p['random_state'], verbose=False
    )
    # Treinando e avaliando a versão do modelo preditivo:
    models[v], train_mtcs, test_mtcs = mllib.train_validate_binary_clf_model(
        classifier=clf, X_train=X_train, y_train=y_train, X_valid=X_test, y_valid=y_test, threshold=THRESHOLD, verbose=False
    )
    # Salvando as variáveis preditoras:
    features[v] = p['cat_variables']+p['num_variables']
    # Salvando as métricas dos dados de treino:
    metrics = metrics.append(
        pd.DataFrame(
            data=[{'Version':v, 'Model':model_label,'Data set':'train','AUC':train_mtcs['auc'],'Accuracy':train_mtcs['accuracy'],
                   'Precision':train_mtcs['precision'],'Recall':train_mtcs['recall'],'F1_score':train_mtcs['f1_score']}]
        ),
        ignore_index=True
    )
    # Salvando as métricas dos dados de teste:
    metrics = metrics.append(
        pd.DataFrame(
            data=[{'Version':v,'Model':model_label,'Data set':'test','AUC':test_mtcs['auc'],'Accuracy':test_mtcs['accuracy'],
                   'Precision':test_mtcs['precision'],'Recall':test_mtcs['recall'],'F1_score':test_mtcs['f1_score']}]
        ),
        ignore_index=True
    )

# Fim do treinamento e avaliação:
ept.end(msg="Tempo gasto:")
print("\nWe trained and evaluated {} predictive models!".format(len(models_version)))

Iniciando o treinamento e avaliação de cada versão do modelo...
Tempo gasto: 0.33 seconds.

We trained and evaluated 16 predictive models!


### Análise das métricas nos dados de `teste`:

In [16]:
# Verificando os modelos com "AUC >= 90%":
metrics.query("`Data set`=='test' and AUC>=0.8").sort_values(by='AUC', ascending=False, ignore_index=True)

,Version,Model,Data set,AUC,Accuracy,Precision,Recall,F1_score
0,v08,DTC,test,0.9000,0.8387,1.0000,0.8,0.8889
1,v07,DTC,test,0.8167,0.8065,0.9524,0.8,0.8696
2,v14,DTC,test,0.8167,0.8065,0.9524,0.8,0.8696
3,v16,DTC,test,0.8167,0.8065,0.9524,0.8,0.8696


In [17]:
# Verificando as métricas de treino e teste da versão "v08":
metrics.query("Version=='v08'").reset_index(drop=True)

,Version,Model,Data set,AUC,Accuracy,Precision,Recall,F1_score
0,v08,DTC,train,1.0,1.0000,1.0,1.0,1.0000
1,v08,DTC,test,0.9,0.8387,1.0,0.8,0.8889


**Análise:** a versão **`v08`** do modelo foi a versão que obteve o melhor desempenho em **todas** as métricas. Entretanto, podemos observar que nos dados de treino ele obteve uma performance **perfeita** conseguindo um desempenho de **100%** em todas as métricas. Ou seja, ocorreu um ajuste excessivo (**overfitting**) nos dados de treino (o modelo aprendeu demais o relacionamento dos dados).

Agora, vamos trabalhar, nesta versão, com a **seleção de variáveis** para tentarmos construir um modelo **menos complexo**.

Para retornar ao ínicio deste **notebook** clique **[aqui](#notebook-header)**.

___
<a name='modelos-selecao-variaveis'></a>
## <font color='blue'>4- Trabalhando com `seleção de variáveis` nos melhores modelos preditivos</font>

In [18]:
# Definindo um dicionário dos modelos que serão criados com os parâmetros de cada versão:
models_version = {
    'v17':{'train_set':'v1','cat_variables':cat_variables, 'num_variables':num_variables,'scaler':RobustScaler(),'balance':True,'random_state':SEED}
}

### Treinando a avaliando cada `versão` do modelo preditivo:

In [19]:
# Inicia o treinamento e avaliação de cada modelo:
ept.start(msg="Iniciando o treinamento e avaliação com seleção de variáveis...")
# Loop para treinar e avaliar cada versão do modelo:
for v, p in models_version.items():
    # Preparando os dados de "treino" (X_train e y_train) e "teste" (X_test e y_test):
    X_train, y_train, X_test, y_test, scaler[v] = mllib.prepare_train_test_data(
    train_set=train_set[p['train_set']], test_set=test_set, target=target_variable, cat_features=p['cat_variables'],
    num_features=p['num_variables'], scaler=p['scaler'], balance=p['balance'], random_state=p['random_state'], verbose=False
    )
    # Treinando e avaliando a versão do modelo preditivo:
    models[v], train_mtcs, test_mtcs = mllib.train_validate_binary_clf_model(
        classifier=RFECV(estimator=clf, step=1, cv=NUM_FOLDS, scoring='roc_auc', n_jobs=-1),
        X_train=X_train, y_train=y_train, X_valid=X_test, y_valid=y_test, threshold=THRESHOLD, verbose=False
    )
    
    # Visualizando as features selecionadas:
    print("\nSelected features: {}".format(models[v].n_features_))
    print(list(models[v].get_feature_names_out(input_features=p['cat_variables']+p['num_variables'])))
    print() # pula 1 linha.
    
    # Salvando as variáveis preditoras:
    features[v] = list(models[v].get_feature_names_out(input_features=p['cat_variables']+p['num_variables']))
    # Salvando as métricas dos dados de treino:
    metrics = metrics.append(
        pd.DataFrame(
            data=[{'Version':v, 'Model':model_label,'Data set':'train','AUC':train_mtcs['auc'],'Accuracy':train_mtcs['accuracy'],
                   'Precision':train_mtcs['precision'],'Recall':train_mtcs['recall'],'F1_score':train_mtcs['f1_score']}]
        ),
        ignore_index=True
    )
    # Salvando as métricas dos dados de teste:
    metrics = metrics.append(
        pd.DataFrame(
            data=[{'Version':v,'Model':model_label,'Data set':'test','AUC':test_mtcs['auc'],'Accuracy':test_mtcs['accuracy'],
                   'Precision':test_mtcs['precision'],'Recall':test_mtcs['recall'],'F1_score':test_mtcs['f1_score']}]
        ),
        ignore_index=True
    )

# Fim do treinamento e avaliação:
ept.end(msg="Tempo gasto:")
print("\nWe trained and evaluated {} predictive models!".format(len(models_version)))

Iniciando o treinamento e avaliação com seleção de variáveis...

Selected features: 1
['Albumin']

Tempo gasto: 1.09 seconds.

We trained and evaluated 1 predictive models!


### Análise das métricas nos dados de `teste`:

In [20]:
# Comparando as duas versões dos modelos:
metrics.query("Version=='v08' or Version=='v17'").reset_index(drop=True)

,Version,Model,Data set,AUC,Accuracy,Precision,Recall,F1_score
0,v08,DTC,train,1.0000,1.0000,1.0000,1.0000,1.0000
1,v08,DTC,test,0.9000,0.8387,1.0000,0.8000,0.8889
2,v17,DTC,train,0.9863,0.9278,0.8952,0.9691,0.9307
3,v17,DTC,test,0.6667,0.8065,0.9130,0.8400,0.8750


**Análise:** a versão **`v17`** obteve um desempenho **ruim** em relação a versão **`v08`**, ou seja, a seleção de variáveis, neste caso, não obteve um resultado satisfatório.

Para retornar ao ínicio deste **notebook** clique **[aqui](#notebook-header)**.

<a name='modelos-conclusao'></a>
## <font color='blue'>5- Concluir e salvar o `melhor modelo preditivo` construído com o *Decision Tree Classifier*</font>

Portanto, o modelo preditivo construido na versão **`v08`** obteve o melhor desempenho na métrica **`AUC`** e também nas outras métricas e por isso, ele foi escolhido como o **melhor modelo preditivo construído com o `DTC`**.

Este modelo foi contruido com as seguintes premissas:
* utilizamos a `versão 1` dos dados de treino;

* utilizamos **todas** as variáveis preditoras;

* aplicamos a **padronização de escala**, utilizando a classe `RobustScaler` nas variáveis numéricas;

* **SIM**, fizemos o **balanceamento** do *dataset* de treino, ou seja, criamos registros sintéticos da classe minoritária (`target = 0`).

### Salvando *todas* as informações do melhor modelo de classificação com `Decision Tree Classifier`:

In [21]:
# Melhor versão do modelo preditivo:
best_version = 'v08'
# Cria dataframes para salvar as melhores métricas de treino e teste:
best_train_mtcs = metrics.query("Version==@best_version and `Data set`=='train'").reset_index(drop=True)
best_test_mtcs = metrics.query("Version==@best_version and `Data set`=='test'").reset_index(drop=True)
# Remove a coluna 'Version':
best_train_mtcs.drop(columns='Version', inplace=True)
best_test_mtcs.drop(columns='Version', inplace=True)
# Salva as melhores métricas criadas com este algoritmo:
models_best_metrics = models_best_metrics.append(best_train_mtcs, ignore_index=True)
models_best_metrics = models_best_metrics.append(best_test_mtcs, ignore_index=True)
# Salva as variáveis preditoras, scaler e modelo da melhor versão:
best_model = {
    'variables': features[best_version],
    'scaler': scaler[best_version],
    'model': models[best_version]
}

In [22]:
# Salvando as métricas dos melhores modelos preditivos:
dslib.pickle_object_save (
    path=ML_PATH, file="models_best_metrics.pkl", object_name=models_best_metrics, msg=None)

In [23]:
# Salvando as informações do melhor modelo preditivo criado com o "DTC":
dslib.pickle_object_save (
    path=ML_PATH, file="dtc_best_model.pkl", object_name=best_model, msg=None)

Para retornar ao ínicio deste **notebook** clique **[aqui](#notebook-header)**.

## <font color='black'>FIM</font>